<a href="https://colab.research.google.com/github/Uniholder/Specialization-Yandex-MIPT/blob/main/6_Final_projects/2_churn_prediction/week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir data
!wget -O data/orange_small_churn_data.train https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
!wget -O data/orange_small_churn_labels.train https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train
!gdown -O data/ 1rW8ZBa870U3DHzERRfy1q777914JKPtA

--2022-08-16 19:15:03--  https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23722578 (23M) [application/octet-stream]
Saving to: ‘data/orange_small_churn_data.train’

data/orange_small_c 100%[===================>]  22.62M  13.4MB/s    in 1.7s    

2022-08-16 19:15:06 (13.4 MB/s) - ‘data/orange_small_churn_data.train’ saved [23722578/23722578]

--2022-08-16 19:15:06--  https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117024 (114K) [application/octet-stream]
Saving to: ‘data/orange_small_churn_labels.train’

data/ora

# Входные данные

In [2]:
import pandas as pd
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

PATH_TO_DATA = 'data'

In [3]:
df_raw = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_data.train'))
labels = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_labels.train'), header=None, squeeze=True)
df_raw.shape

(40000, 230)

In [4]:
num_vars = df_raw.columns[:190]
cat_vars = df_raw.columns[190:]
na_vars = df_raw.columns[df_raw.isna().all()]

nna_num_vars = list(set(num_vars) - set(na_vars))
nna_cat_vars = list(set(cat_vars) - set(na_vars))

In [5]:
df_raw['target'] = labels
df = df_raw.dropna(axis=1, how='all')
df.shape

(40000, 213)

Отделим hold-out dataset

In [6]:
df_train, df_ho = train_test_split(df, test_size=5000)
df_train.shape, df_ho.shape

((35000, 213), (5000, 213))

In [ ]:
# df_ho.to_csv(os.path.join(PATH_TO_DATA, 'hold_out_dataset.csv'))

Preprocessing

In [7]:
cols = df_train.drop('target', axis=1).columns

In [8]:
cat_encoder = ColumnTransformer(
    transformers=[
        ("cat", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), nna_cat_vars)
    ], remainder='passthrough'
)

preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cat_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

In [9]:
%%time
df_train_enc = pd.DataFrame(preprocessor.fit_transform(df_train[cols]), columns=cols)

CPU times: user 6.9 s, sys: 629 ms, total: 7.52 s
Wall time: 7.56 s


In [10]:
df_train_enc

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,0.0,109.0,1.0,0.0,0.0,0.0,...,1843.0,3.0,3098.0,0.0,0.0,0.0,14.0,4.0,8.0,1.0
1,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,1609.0,4.0,3447.0,0.0,0.0,0.0,7.0,2.0,8.0,0.0
2,0.0,0.0,0.0,0.0,0.0,330.0,1.0,0.0,0.0,0.0,...,1278.0,4.0,3562.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0
3,0.0,0.0,0.0,0.0,0.0,241.0,1.0,0.0,0.0,0.0,...,1212.0,4.0,27.0,0.0,0.0,0.0,18.0,2.0,8.0,1.0
4,0.0,0.0,0.0,0.0,0.0,122.0,2.0,0.0,0.0,0.0,...,2898.0,4.0,2332.0,0.0,0.0,0.0,10.0,2.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,0.0,0.0,0.0,0.0,0.0,427.0,2.0,0.0,0.0,0.0,...,3107.0,4.0,1078.0,3.0,0.0,0.0,8.0,2.0,8.0,0.0
34996,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,...,1476.0,4.0,358.0,0.0,0.0,0.0,1.0,2.0,8.0,0.0
34997,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1249.0,4.0,188.0,0.0,0.0,0.0,10.0,2.0,8.0,0.0
34998,0.0,0.0,0.0,0.0,0.0,118.0,2.0,0.0,0.0,0.0,...,2397.0,4.0,2199.0,0.0,0.0,1.0,1.0,2.0,8.0,0.0


In [11]:
df_train_enc['Var229'].value_counts()

0.0    28027
1.0     6922
2.0       27
3.0       24
Name: Var229, dtype: int64

# Модель

In [12]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 79 kB/s 


In [40]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, cross_validate
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report

from catboost import CatBoostClassifier, cv, Pool

In [14]:
X, y = df_train_enc, df_train['target']
cv_scheme = StratifiedShuffleSplit(random_state=0, )

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

## LogisticRegression

In [16]:
cv_results = cross_validate(estimator=LogisticRegression(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

In [17]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.57733742, 0.54004853, 0.56236039, 0.53437318, 0.58144502,
        0.56569723, 0.54352236, 0.57357649, 0.5571973 , 0.58503203]),
 0.562058995567172)

In [18]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
print(classification_report(y_val, lr.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.92      1.00      0.96      6473
           1       0.00      0.00      0.00       527

    accuracy                           0.92      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.92      0.89      7000



## RidgeClassifier

In [19]:
cv_results = cross_validate(estimator=RidgeClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

In [20]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.6605688 , 0.69325569, 0.66629538, 0.67257353, 0.68174892,
        0.66298356, 0.63471044, 0.6475872 , 0.63357989, 0.67444268]),
 0.6627746095811508)

In [21]:
rc = RidgeClassifier(random_state=0)
rc.fit(X_train, y_train)
print(classification_report(y_val, rc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.92      1.00      0.96      6473
           1       0.00      0.00      0.00       527

    accuracy                           0.92      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.92      0.89      7000



## SGDClassifier

In [22]:
%%time
cv_results = cross_validate(estimator=SGDClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

In [23]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.52819152, 0.49981833, 0.57863951, 0.45887691, 0.49635879,
        0.54119337, 0.56296438, 0.52134989, 0.48579553, 0.58529054]),
 0.5258478781156967)

In [24]:
sgdc = SGDClassifier(random_state=0)
sgdc.fit(X_train, y_train)
print(classification_report(y_val, sgdc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.92      1.00      0.96      6473
           1       0.00      0.00      0.00       527

    accuracy                           0.92      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.92      0.89      7000



### SGDClassifier log loss

In [25]:
%%time
cv_results = cross_validate(estimator=SGDClassifier(random_state=0, loss='log'), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

CPU times: user 427 ms, sys: 77.2 ms, total: 504 ms
Wall time: 43.1 s


In [26]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.50959533, 0.50251186, 0.48302695, 0.49142443, 0.58730503,
        0.5563479 , 0.50159456, 0.47394329, 0.48207153, 0.54885105]),
 0.5136671912358428)

In [27]:
sgdc = SGDClassifier(random_state=0, loss='log')
sgdc.fit(X_train, y_train)
print(classification_report(y_val, sgdc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.92      1.00      0.96      6473
           1       0.00      0.00      0.00       527

    accuracy                           0.92      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.92      0.89      7000



## RandomForestClassifier

In [28]:
rfc = RandomForestClassifier(oob_score=True)

In [29]:
rfc.fit(X, y)

RandomForestClassifier(oob_score=True)

In [30]:
rfc.oob_score_

0.9258857142857143

In [31]:
%%time
cv_results = cross_validate(estimator=RandomForestClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

CPU times: user 690 ms, sys: 96.9 ms, total: 787 ms
Wall time: 1min 29s


In [32]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.65455095, 0.65374801, 0.64062167, 0.65992252, 0.67668471,
        0.64420391, 0.65415424, 0.66073141, 0.6671418 , 0.66583256]),
 0.6577591762874083)

In [33]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
print(classification_report(y_val, rfc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.92      1.00      0.96      6473
           1       0.00      0.00      0.00       527

    accuracy                           0.92      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.92      0.89      7000



## GradientBoostingClassifier

In [34]:
%%time
cv_results = cross_validate(estimator=GradientBoostingClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

CPU times: user 2.64 s, sys: 328 ms, total: 2.97 s
Wall time: 6min 58s


In [35]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.7236124 , 0.73969019, 0.73720752, 0.74452746, 0.74806146,
        0.72909894, 0.72583299, 0.72651918, 0.71050334, 0.73503399]),
 0.7320087465258709)

In [36]:
gbc = GradientBoostingClassifier(random_state=0)
gbc.fit(X_train, y_train)
print(classification_report(y_val, gbc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6473
           1       0.50      0.01      0.01       527

    accuracy                           0.92      7000
   macro avg       0.71      0.50      0.49      7000
weighted avg       0.89      0.92      0.89      7000



## CatBoost

In [37]:
base_params = {
    'eval_metric': 'AUC:hints=skip_train~false', 
    'auto_class_weights': 'Balanced', 
    'verbose': False,
    'loss_function': 'Logloss'
}

In [42]:
cv_results = cv(
    params={**base_params},
    pool=Pool(X, y),
    fold_count=5,
    shuffle=True,
    # plot=True,
    type='Classical'
)

Training on fold [0/5]

bestTest = 0.7106812462
bestIteration = 423

Training on fold [1/5]

bestTest = 0.7379935782
bestIteration = 413

Training on fold [2/5]

bestTest = 0.7085311388
bestIteration = 218

Training on fold [3/5]

bestTest = 0.7228578458
bestIteration = 405

Training on fold [4/5]

bestTest = 0.7367928468
bestIteration = 444



In [43]:
cv_results

,iterations,test-AUC-mean,test-AUC-std,train-AUC-mean,train-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.657860,0.015688,0.660945,0.011773,0.690081,0.000203,0.689972,0.000243
1,1,0.674257,0.014363,0.682398,0.006114,0.687001,0.000379,0.686699,0.000336
2,2,0.680459,0.014780,0.689773,0.006310,0.684428,0.000523,0.683962,0.000514
3,3,0.682848,0.017237,0.695171,0.006502,0.681902,0.000734,0.681106,0.000671
4,4,0.686323,0.018015,0.697607,0.007964,0.679395,0.001037,0.678506,0.000801
...,...,...,...,...,...,...,...,...,...
995,995,0.711461,0.011689,0.966103,0.001242,0.663224,0.014082,0.368113,0.002468
996,996,0.711388,0.011643,0.966192,0.001237,0.663376,0.014073,0.367912,0.002502
997,997,0.711373,0.011671,0.966297,0.001197,0.663474,0.014078,0.367692,0.002525
998,998,0.711374,0.011655,0.966373,0.001170,0.663555,0.014128,0.367523,0.002511


In [44]:
cv_results['test-AUC-mean'].argmax(), cv_results['test-AUC-mean'].max()

(413, 0.7224084040788601)

In [45]:
cb_classifier = CatBoostClassifier(**base_params)
# cb_classifier = CatBoostClassifier(**base_params)
# cb_classifier = CatBoostClassifier(**base_params)
cb_classifier.fit(X, y)

# Тест

In [61]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(predicted_labels.shape[0]),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [47]:
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_test_data.csv'), index_col=0)
df_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,1225.0,7.0,NaN,NaN,NaN,...,zCkv,APgdzOv,jySVZNlOJy,NaN,ELof,xb3V,6fzt,Zy3gnGM,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,896.0,14.0,NaN,NaN,NaN,...,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,NaN,...,oslk,6YSocsg,LM8l689qOp,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,2296.0,7.0,NaN,NaN,NaN,...,oslk,5nQ7A2G,jySVZNlOJy,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c,NaN
4,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,...,oslk,MI8s5nE,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,714.0,7.0,NaN,NaN,NaN,...,zCkv,P6pu4Vl,LM8l689qOp,NaN,ELof,xb3V,ZI9m,R4y5gQQWY8OodqDV,am7c,NaN
9996,NaN,NaN,NaN,NaN,NaN,812.0,7.0,NaN,NaN,NaN,...,oslk,sXbT3Cb,LM8l689qOp,NaN,ELof,WqMG,RAYp,55YFVY9,mj86,NaN
9997,NaN,NaN,NaN,NaN,NaN,819.0,7.0,NaN,NaN,NaN,...,oslk,05jAV0N,M_8D,NaN,ELof,TNEC,RAYp,55YFVY9,am7c,NaN


In [48]:
df_test.drop(na_vars, axis=1, inplace=True)

In [49]:
df_test_enc = pd.DataFrame(preprocessor.transform(df_test), columns=cols)

In [52]:
submission = cb_classifier.predict_proba(df_test_enc)[:, 1]

In [62]:
write_to_submission_file(submission, 'submission.csv', target='result', index_label='ID')